In [ ]:
import pandas as pd 
import numpy as np
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import  MinMaxScaler #stardadizzo i valori a intervallo [0,1]
from sklearn.preprocessing import  StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from collections import Counter
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt

In [ ]:
input = pd.read_csv("/content/drive/My Drive/dati/dati_experiment/file2.csv") 
data_obj = pd.DataFrame(input)
data_obj_droped = data_obj.drop(columns=['PSA', 'ETA','STATUS','Tipologia'])

col_status = data_obj['STATUS']
col_typology = data_obj['Tipologia']

# create a dictonary 
data = {"STATUS": col_status, "Tipologia": col_typology} 
data_class = pd.concat(data, axis = 1) 

# show the dataframe 
tuple_class = list(map(tuple, data_class.to_numpy())) #m=2, t=1, n=0
#print(len(tuple_class))  #45 sample
encode_tuple = { (0,0) : 1, (0,1) : 2, (0,2) : 3, (1,0) : 4, (1,1) : 5, (1,2) : 6}
macro_class = []
for couple in tuple_class: 
  #print(encode_tuple[couple])
  macro_class.append(encode_tuple[couple])

#print(macro_class)
series_macro_class = pd.Series((v for v in macro_class) )
#print(series_macro_class)
print("NUMERO DI TUPLE PER CLASSE MACRO:")
series_macro_class.value_counts()

In [ ]:
n_fold = 5
fold_obj = StratifiedKFold(n_fold ,  True, random_state = 4)  # 5 fold, shuffle=true
print(fold_obj)
print(len(data_obj_droped)) #45
y = data_obj['STATUS']

#Strutture utili per immagazzinamento risultati degli esperimenti
#n = data_objs_oversampling #.to_numpy()
n = data_obj_droped
test_predicted_logit = np.zeros((np.array(n).shape[0],))
test_predicted_svc = np.zeros((np.array(n).shape[0],))
test_predicted_bayes = np.zeros((np.array(n).shape[0],))
test_predicted_clf =  np.zeros((np.array(n).shape[0],))
test_predicted_knn =  np.zeros((np.array(n).shape[0],))

#STANDARDIZZAZIONE
#StandardScaler(with_std=True) o #MinMaxScaler()
scaler = MinMaxScaler()
scaler.fit(data_obj_droped) # data_obj_droped: dataframe senza le colonne: PSA ETA  STATUS Tipologia
data_objs = scaler.transform(data_obj_droped) 
data_objs

#funzione oversampling "normale"

In [ ]:
'''
#Funzione per fare oversampling solo sul training (nella fase di K-fold split)
def oversampling_training(dataset_training, y_status, y_tipology): 
  data = {"STATUS": y_status, "Tipologia": y_tipology} 
  data_class = pd.concat(data, axis = 1) 
  tuple_class = list(map(tuple, data_class.to_numpy()))
  #print(len(tuple_class))  #45 sample
  encode_tuple = { (0,0) : 1, (0,1) : 2, (0,2) : 3, (1,0) : 4, (1,1) : 5, (1,2) : 6}
  macro_class = []
  for couple in tuple_class: 
    macro_class.append(encode_tuple[couple])

  series_macro_class = pd.Series((v for v in macro_class) )
  #print(series_macro_class)
  #print("NUMERO DI TUPLE PER CLASSE MACRO:")
  #print(series_macro_class.value_counts())

  #oversampling "NORMALE"                                
  oversample = SMOTE(k_neighbors=1)#Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3
  data_objs_oversampling, macro_class_oversampling = oversample.fit_resample(dataset_training, series_macro_class)

  macro= pd.Series((v for v in macro_class_oversampling) )
  #print(series_macro_class) 
  print("NUMERO DI TUPLE PER CLASSE MACRO:")
  print(macro.value_counts())
  #DALL'OVERSAMPLING DELLA MACRO_CLASS DEVO CODIFICARE LE NUOVE ETICHETTE
  micro_class = list()
  for encoded in macro_class_oversampling: 
    #print(encoded)
    for key, value in encode_tuple.items():  #  in dictionary.iteritems():  (for Python 2.x)
      if encoded == value:
          #print(key)
          micro_class.append(key)
  #print(micro_class)
  new_df = pd.DataFrame(micro_class, columns=['STATUS', 'Tipologia'])
  #print(new_df)
  y=new_df['STATUS']
  #print(y)
  #print("Nuova lunghezza del vettore target y (dopo l'oversampling): ", len(y))
  return data_objs_oversampling, y
'''

#funzione oversampling "alternativo"

In [ ]:
#Funzione per fare oversampling solo sul training (nella fase di K-fold split)
def oversampling_training(dataset_training, y_status, y_tipology): 
  data = {"STATUS": y_status, "Tipologia": y_tipology} 
  data_class = pd.concat(data, axis = 1) 
  #show the dataframe 
  #print(data_class)
  tuple_class = list(map(tuple, data_class.to_numpy()))
  #print(len(tuple_class))  #45 sample
  encode_tuple = { (0,0) : 1, (0,1) : 2, (0,2) : 3, (1,0) : 4, (1,1) : 5, (1,2) : 6}
  macro_class = []
  for couple in tuple_class: 
    #print(encode_tuple[couple])
    macro_class.append(encode_tuple[couple])

  series_macro_class = pd.Series((v for v in macro_class) )
 
  print("NUMERO DI TUPLE PER CLASSE MACRO:")
  print(series_macro_class.value_counts())  #<----vedi problema qui: #Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 3
  table_count = series_macro_class.value_counts()
  table_count = table_count.to_dict()
 
  n_sintetic = 0
  #oversampling alternativo                  #FACCIO OVERSAMPLING SOLO DELLE SOTTOCLASSI DEBOLI                                     
  oversample = SMOTE({1: table_count[1]+ n_sintetic ,2: table_count[2]+ n_sintetic , 6: table_count[6]+ n_sintetic,
                      3: table_count[6]+n_sintetic , 4: table_count[1]+n_sintetic, 5: table_count[2]+n_sintetic}, k_neighbors=1) 
    
 
  data_objs_oversampling, macro_class_oversampling = oversample.fit_resample(dataset_training, series_macro_class)

  macro= pd.Series((v for v in macro_class_oversampling) )
  print("NUMERO DI TUPLE PER CLASSE MACRO:")
  print(macro.value_counts())
  #DALL'OVERSAMPLING DELLA MACRO_CLASS DEVO CODIFICARE LE NUOVE ETICHETTE
  micro_class = list()
  for encoded in macro_class_oversampling: 
    #print(encoded)
    for key, value in encode_tuple.items():  #  in dictionary.iteritems():  (for Python 2.x)
      if encoded == value:
          #print(key)
          micro_class.append(key)
  #print(micro_class)
  new_df = pd.DataFrame(micro_class, columns=['STATUS', 'Tipologia'])
  #print(new_df)
  y=new_df['STATUS']
  #print(y)
  #print("Nuova lunghezza del vettore target y (dopo l'oversampling): ", len(y))
  return data_objs_oversampling, y

In [ ]:
import itertools
#Evaluation of Model - Confusion Matrix Plot
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center")
                # color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
#from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

i=0
#AGGIUNGERE PRECISION, RECALL, E MATRICI DI CONFUSIONE
dict_model = { "logit": {"accuracy": [], "precision": [], "recall": []},
                 "svm": {"accuracy": [], "precision": [], "recall": []},
                 "rf": {"accuracy":  [], "precision": [], "recall": []}, 
                 "nb": {"accuracy":  [], "precision": [], "recall": []},
                 "knn": {"accuracy":  [], "precision": [], "recall": []},
              }

#variance = [95, 90, 80, 70, 60]
# Riduzione feature
#for var_val in variance:   #se vogliamo provare a mantenere una percentuale diversa di varianza tra i dati
var_val = 95  
pca = PCA(float("0."+ str(var_val) ))   #manteniamo un numero di componenti tale che la varianza venga  mantenuta al 95%
pca.fit(data_objs)
print(pca.explained_variance_)
print("Varianza mantenuta:  %", str(var_val))
print("Componenti Pricipali: ", len(pca.components_))

df = pd.DataFrame({'var':pca.explained_variance_, 'PC': [str(i+1) for i in range(0, len(pca.explained_variance_))]})
sns.barplot(x='PC', y='var', data=df, color="b")

for train_idx, test_idx in fold_obj.split(data_objs, y):  
      i += 1
     
      X_train, X_test = data_objs[train_idx, :], data_objs[test_idx, :]
      y_train, y_test = y[train_idx], y[test_idx]

      dataset_training = data_objs[train_idx, :]
      y_status = y[train_idx]
      y_tipology = col_typology[train_idx]
      
          
      #fare oversampling su X_train e y_train
      #oversampling_training(dataset_training, y_status, y_tipology) #Test only function

      X_oversampled, y_oversampled = oversampling_training(dataset_training, y_status, y_tipology)

      #print("X_train oversampled", len(X_oversampled))
      #print("y_train oversampled", len(y_oversampled))

      X_train = X_oversampled
      y_train = y_oversampled

      print("TRAINING sample: ", len(X_train))
      print("TEST sample: ", len(X_test))

      # scatter plot of examples by class label
      #BUILD Logit
      logisticRegr = LogisticRegression(solver = 'lbfgs')  # handle L2 or no penalty
      logisticRegr.fit(X_train, y_train)
      test_predicted_logit[test_idx]  = logisticRegr.predict(X_test)
      print("Fold: " + str(i))
      #print("Mean Accuracy Logistic Regression:  %", logisticRegr.score(test_rad,y_test)*100)

  
      #BUILD SVC
      model_svc = SVC(kernel='rbf')
      model_svc.fit(X_train, y_train) 
      test_predicted_svc[test_idx] = model_svc.predict(X_test)
      
      #print("Fold: " + str(i))
      #print("\t Accuracy SVM: %", accuracy_score(y_test, test_predicted_svc[test_idx])*100)

      #BUILD RANDOM FOREST 
      clf = RandomForestClassifier(max_depth=None, random_state=0)
      clf.fit(X_train, y_train) 
      test_predicted_clf[test_idx] = clf.predict(X_test)
      #print("\t Accuracy RF:  %", accuracy_score(y_test, test_predicted_clf[test_idx])*100) 
             
      #tn, fp, fn, tp = confusion_matrix(y_test, test_predicted_clf[test_idx]).ravel()
      #print("TP rf ", tp, "TN rf:", tn, "FP rf ", fp, "FN rf ", fn)  
      
      from sklearn.naive_bayes import GaussianNB
      #BUILD NAIVE BAYESIAN 
      model_naiveBay = GaussianNB()
      model_naiveBay.fit(X_train, y_train)
      test_predicted_bayes[test_idx] = model_naiveBay.predict(X_test)
      #print("\t Accuracy NB:  %", accuracy_score(y_test, test_predicted_bayes[test_idx])*100)

      #BUILD KNN
      neigh = KNeighborsClassifier(n_neighbors=5)
      neigh.fit(X_train, y_train)
      test_predicted_knn[test_idx] = neigh.predict(X_test)
      

print("End of kfold") 

# Compute confusion matrix
label = ["logit", "svm", "nb", "rf", "knn"]  
dict_pred = {  "logit": {"y_pred": []},
                 "svm": {"y_pred": []},
                 "rf":  {"y_pred": []},
                 "nb":  {"y_pred": []},
                 "knn":  {"y_pred": []}, 
              }
  #SVM
dict_pred['svm']['y_pred'].append(test_predicted_svc)
  #Logit 
dict_pred['logit']['y_pred'].append(test_predicted_logit)
  #Rf
dict_pred['rf']['y_pred'].append(test_predicted_clf)
  #Nb
dict_pred['nb']['y_pred'].append(test_predicted_bayes)
  #Nb
dict_pred['knn']['y_pred'].append(test_predicted_knn)

for name in label: 
  y_pred = dict_pred[name]['y_pred']
  print(y_pred[0])
  cnf_matrix = confusion_matrix(y, y_pred[0])
  np.set_printoptions(precision=2)
  print(cnf_matrix)
  plt.figure()
  tit = 'Confusion matrix with MinMaxScaler -' + name 
  plot_confusion_matrix(cnf_matrix, classes=['0','1'],
                        title= tit)
  #plt.savefig('/content/drive/My Drive/dati/last_test/Plot/' + tit +'.png')


print("\n\nRisultati metriche: \n")

print("Accuracy SVM: ", accuracy_score(y,test_predicted_svc))
print("Accuracy RF: ", accuracy_score(y,test_predicted_clf))
print("Accuracy NB: ", accuracy_score(y,test_predicted_bayes))
print("Accuracy LOGIT: ", accuracy_score(y,test_predicted_logit))
print("Accuracy KNN: ", accuracy_score(y,test_predicted_knn))

print("\nPrecision SVM: ", precision_score(y,test_predicted_svc))
print("Precision RF: ", precision_score(y,test_predicted_clf))
print("Precision NB: ", precision_score(y,test_predicted_bayes))
print("Precision LOGIT: ", precision_score(y,test_predicted_logit))
print("Precision KNN: ", precision_score(y,test_predicted_knn))

print("\nRecall SVM: ", recall_score(y,test_predicted_svc))
print("Recall RF: ", recall_score(y,test_predicted_clf))
print("Recall NB: ", recall_score(y,test_predicted_bayes))
print("Recall LOGIT: ", recall_score(y,test_predicted_logit))
print("Recall KNN: ", recall_score(y,test_predicted_knn))

#plot_confusion_matrix(clf, data_objs, y)  # doctest: +SKIP
#plt.show() 

  #SVM
dict_model['svm']['accuracy'].append(accuracy_score(y, test_predicted_svc))
  #Logit 
dict_model['logit']['accuracy'].append(accuracy_score(y, test_predicted_logit))
  #Rf
dict_model['rf']['accuracy'].append(accuracy_score(y, test_predicted_clf))
  #Nb
dict_model['nb']['accuracy'].append(accuracy_score(y, test_predicted_bayes))
  #KNN
dict_model['knn']['accuracy'].append(accuracy_score(y, test_predicted_knn))


  #SVM
dict_model['svm']['precision'].append(precision_score(y, test_predicted_svc))
  #Logit 
dict_model['logit']['precision'].append(precision_score(y, test_predicted_logit))
  #Rf
dict_model['rf']['precision'].append(precision_score(y, test_predicted_clf))
  #Nb
dict_model['nb']['precision'].append(precision_score(y, test_predicted_bayes))
  #KNN
dict_model['knn']['precision'].append(precision_score(y, test_predicted_knn))



  #SVM
dict_model['svm']['recall'].append(recall_score(y, test_predicted_svc))
  #Logit 
dict_model['logit']['recall'].append(recall_score(y, test_predicted_logit))
  #Rf
dict_model['rf']['recall'].append(recall_score(y, test_predicted_clf))
  #Nb
dict_model['nb']['recall'].append(recall_score(y, test_predicted_bayes))
  #KNN
dict_model['knn']['precision'].append(recall_score(y, test_predicted_knn))

#j=0

'''
for key in dict_model.keys():   
    #print(j)
    np.save('/content/drive/My Drive/dati/last_test/StandardScaler/NO_PCA/accuracy_'+ str(key)+ '_NO_PCA' + str(n_fold) + '-fold', list(dict_model[key]['accuracy']))
    #np.savetxt('/content/drive/My Drive/dati/last_test/StandardScaler/NO_PCA/accuracy_'+ str(key)+ '_NO_PCA' + str(n_fold) + '-fold.txt', list(dict_model[key]['accuracy'])) 
    np.save('/content/drive/My Drive/dati/last_test/StandardScaler/NO_PCA/precision_'+ str(key)+ '_NO_PCA' + str(n_fold) + '-fold', list(dict_model[key]['precision']))
    np.save('/content/drive/My Drive/dati/last_test/StandardScaler/NO_PCA/recall_'+ str(key)+ '_NO_PCA' + str(n_fold) + '-fold', list(dict_model[key]['recall']))
    #j+=1
'''